# 把已经添加了一些列的PLC数据转换为1min的总结


In [7]:
#探索PLC文件不同分钟的细节
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt

import os.path
import datetime
#训练文件夹
TrainFolder='H:\\12_Kaggle\\toollife\\pythonProject\\PreProcessed'
TrainFile=os.path.join(TrainFolder,'PLC_All.csv')
# TrainFolder='H:\\12_Kaggle\\toollife\\pythonProject\\TestPreProcessed'
dstfile=os.path.join(TrainFolder,'PLC_All_1s_Feature.csv')

In [137]:
#从PLC_ALL中去掉已经确定的outlier

df_all=pd.read_csv(TrainFile)

In [149]:
outlierlist=[[0,1],
             [0,2],
             [0,13],
             [0,29],
             [1,1],
             [1,8],
             [1,14],
             [1,21],
             [1,25],
             [1,34],
             [1,35],
             [1,42],
             [2,1],
             [2,8],
             [2,20],
             [2,30],
             [2,37]]

In [150]:
newdf=pd.DataFrame()

In [151]:
newdf

""


In [154]:
if [0,4] in outlierlist:
    print('x')

In [156]:
for m in range(0,3):
    dfm=df_all[df_all['Machine']==m]
    csv_count=dfm.csv_no.max()
    
    for csv in range(1,csv_count):
        
        if [m,csv] in outlierlist:
            continue
        else:
            dfc=dfm[dfm['csv_no']==csv]
            newdf=pd.concat([newdf,dfc])
        
        

In [165]:
newdf.to_csv('H:\\12_Kaggle\\toollife\\pythonProject\\PreProcessed\\PLC_All_ORemoved_1213.csv')

In [168]:
df_all.count()[0],newdf.count()[0]

(303159, 264808)

In [163]:
t=newdf[newdf['Machine']==1]
tt=t[t['csv_no']==14]
tt

,Unnamed: 0,Unnamed: 0.1,time,spindle_load,x,y,z,csv_no,x_displacement,y_displacement,z_displacement,displacement,x_move,y_move,z_move,Machine


In [70]:
#plc数据33hz
interval=50

In [77]:
#一些函数
def timediff(x1,x2): #两个时间的时间差
    return (datetime.datetime.strptime(x1,"%H:%M:%S:%f")-datetime.datetime.strptime(x2,"%H:%M:%S:%f")).total_seconds()

#一个df中最大最小的时间差，df要按顺序排好
def df_timediff(df):
    
    t_end=df.loc[df.index.max(),'time']
    t_start=df.loc[df.index.min(),'time']
    diff=timediff(t_start,t_end)
    return diff

In [6]:
dflist=[]
for n in range(0,5):
#     filename="PLC_Processed_0"+str(n+1)+".csv"
    filename=str(n+1)+'.csv'
    
    filepath=os.path.join(TrainFolder,filename)
    
    df=pd.read_csv(filepath)
    df['Machine']=n
    dflist.append(df)

df_all=pd.concat(dflist)

FileNotFoundError: File b'H:\\12_Kaggle\\toollife\\pythonProject\\PreProcessed\\1.csv' does not exist

In [8]:
df_all=pd.read_csv(TrainFile)

In [10]:
df_all.columns,df_all.count()[0]

(Index(['Unnamed: 0', 'Unnamed: 0.1', 'time', 'spindle_load', 'x', 'y', 'z',
        'csv_no', 'x_displacement', 'y_displacement', 'z_displacement',
        'displacement', 'x_move', 'y_move', 'z_move', 'Machine'],
       dtype='object'), 303159)

In [11]:
df_m=df_all[df_all['Machine']==2]
df_m['csv_no'].max()

37

In [50]:
#先1,1
df1=df_all[df_all['Machine']==0]
df1_1=df1[df1['csv_no']==1]

4371

In [89]:
#总时间长度，查看一下所有文件
timelist=[]
for m in range(0,3):
    dfm=df_all[df_all['Machine']==m]
    csv_count=dfm.csv_no.max()
    
    for csv in range(1,csv_count):
        
        dfc=dfm[dfm['csv_no']==csv]
        tf=df_timediff(dfc)
        timelist.append(tf)
        
        if abs(float(tf))<=59:
            print(m,csv)
timelist

0 1
0 29
1 14
1 21
1 25
1 34
1 42
2 1
2 8


In [62]:
#遍历，从开始算0秒，到差距>=1s，作为一个df，然后计算feature

for m in range(0,3):
    dfm=df_all[df_all['Machine']==m]
    csv_count=dfm.csv_no.max()
    
    for csv in range(1,csv_count):
        
        dfc=dfm[dfm['csv_no']==csv]
        
        t_start=df.loc[df.index.min(),'time']
        

In [49]:
diflist

[-1.01,
 -1.16,
 -1.28,
 -1.24,
 -1.29,
 -1.21,
 -1.29,
 -1.963,
 -0.797,
 -1.22,
 -2.183,
 -0.527,
 -1.3,
 -1.24,
 -1.31,
 -1.41,
 -1.26,
 -1.32,
 -1.241,
 -1.3,
 -2.202,
 -0.438,
 -1.29,
 -1.35,
 -1.35,
 -1.27,
 -1.26,
 -1.36,
 -1.37,
 -1.26,
 -1.32,
 -1.24,
 -1.28,
 -1.48,
 -1.19,
 -1.28,
 -1.25,
 -1.36,
 -1.37,
 -1.28]

In [29]:
#每一个csv_no文件的feature包括
#时间差
#spindle: max,min,std,25%,75%,min
#x_displacement: max, min, sum(abs())   and y and z
#displacement: max,min, sum()
#x_move, count, y_move, count, z_move, count
#，y总位移，z总位移，x最大位移，y最大位移，z最大位移，总最大位移，x_move次数，
#

#版本1s需要做每一s的数据

#

dflist=[]
for m in range(0,5):#机器循环
    
    df_m=df_all[df_all['Machine']==m]
    csvmax=df_m['csv_no'].max()
    
    df2=pd.DataFrame()
    print(m,csvmax)
    for csv in range(csvmax): #每一个csv的循环
        
        csvno=csv+1
        
        df=df_m[df_m['csv_no']==csvno]
        
        tmax=datetime.datetime.strptime(df['time'].max(),"%H:%M:%S:%f")
        tmin=datetime.datetime.strptime(df['time'].min(),"%H:%M:%S:%f")

        td=tmax-tmin
        duration=td.total_seconds()
        
        [c,spindle_load_mean,sl_std,sl_min,sl_25,sl_50,sl_75,sl_max]=df['spindle_load'].describe()

        x_disp_max=df['x_displacement'].abs().max()
        x_disp_sum=df['x_displacement'].abs().sum()
        x_disp_mean=df[df['x_displacement']!=0].x_displacement.abs().mean()
        
        y_disp_max=df['y_displacement'].abs().max()
        y_disp_sum=df['y_displacement'].abs().sum()
        y_disp_mean=df[df['y_displacement']!=0].y_displacement.abs().mean()
        
        z_disp_max=df['z_displacement'].abs().max()
        z_disp_sum=df['z_displacement'].abs().sum()
        z_disp_mean=df[df['z_displacement']!=0].z_displacement.abs().mean()
        
        x_move_count=df[df['x_move']!=0].count()[0]
        y_move_count=df[df['y_move']!=0].count()[0]
        z_move_count=df[df['z_move']!=0].count()[0]
        
        all_displacement=df['displacement'].sum()
        
        #写入新的df
        
#         print(df2)


    
        df2.loc[csvno,'Machine']=m
        df2.loc[csvno,'csv_no']=csvno
        df2.loc[csvno,'Duration']=duration
        
        df2.loc[csvno,'sl_mean']=spindle_load_mean
        df2.loc[csvno,'sl_std']=sl_std
        df2.loc[csvno,'sl_25']=sl_25
        df2.loc[csvno,'sl_50']=sl_50
        df2.loc[csvno,'sl_75']=sl_75
        df2.loc[csvno,'sl_max']=sl_max
        
        df2.loc[csvno,'x_disp_max']=x_disp_max
        df2.loc[csvno,'x_disp_mean']=x_disp_mean
        df2.loc[csvno,'x_disp_sum']=x_disp_sum
        
        df2.loc[csvno,'y_disp_max']=y_disp_max
        df2.loc[csvno,'y_disp_mean']=y_disp_mean
        df2.loc[csvno,'y_disp_sum']=y_disp_sum
        
        df2.loc[csvno,'z_disp_max']=z_disp_max
        df2.loc[csvno,'z_disp_mean']=z_disp_mean
        df2.loc[csvno,'z_disp_sum']=z_disp_sum
        
        df2.loc[csvno,'x_move_count']=x_move_count
        df2.loc[csvno,'y_move_count']=y_move_count
        df2.loc[csvno,'z_move_count']=z_move_count
        df2.loc[csvno,'displacement_all']=all_displacement
    dflist.append(df2)

0 10
1 10
2 10
3 10
4 10


In [30]:
dfnew=pd.concat(dflist)
dfnew.to_csv(dstfile)

In [166]:
dfnew

,Machine,csv_no,sl_mean,sl_std,sl_25,sl_50,sl_75,sl_max,x_disp_max,x_disp_mean,x_disp_sum,y_disp_max,y_disp_mean,y_disp_sum,z_disp_max,z_disp_mean,z_disp_sum,x_move_count,y_move_count,z_move_count
1,0.0,1.0,0.914081,2.760115,0.015259,0.033570,0.057985,36.545915,52.0000,20.377945,774.3619,13.8178,6.505908,253.7304,52.0000,10.488883,503.4664,38.0,39.0,48.0
2,0.0,2.0,13.331341,3.384573,11.178930,13.324381,15.164647,24.005860,43.3333,16.478999,3872.5647,43.2932,16.266769,3887.7577,0.0000,NaN,0.0000,235.0,239.0,0.0
3,0.0,3.0,13.701669,3.460452,11.081271,13.394574,15.994751,24.573504,44.6666,17.489855,4005.1769,40.8135,15.326437,3571.0599,0.0000,NaN,0.0000,229.0,233.0,0.0
4,0.0,4.0,13.355145,3.253814,11.105686,13.171789,15.286721,25.382244,57.3324,16.879777,3899.2286,43.3329,16.704484,3942.2583,0.0000,NaN,0.0000,231.0,236.0,0.0
5,0.0,5.0,13.851032,3.598315,11.578722,13.882870,15.814692,25.693533,44.6667,14.893245,3514.8059,42.0042,18.105812,4327.2891,0.0000,NaN,0.0000,236.0,239.0,0.0
6,0.0,6.0,13.781556,4.403636,11.169774,13.763848,16.425062,26.862392,62.7828,13.167179,3081.1199,49.3333,18.466946,4395.1332,2.3252,0.373286,5.2260,234.0,238.0,14.0
7,0.0,7.0,14.041365,4.727645,10.589923,14.160588,17.084262,26.670125,55.9447,12.489778,2935.0978,47.9820,18.342483,4292.1410,2.3384,0.373286,5.2260,235.0,234.0,14.0
8,0.0,8.0,14.185480,4.655623,10.852382,14.056825,17.529832,26.743370,46.0000,14.293632,3216.0673,65.4716,17.436806,3975.5918,2.3206,0.373286,5.2260,225.0,228.0,14.0
9,0.0,9.0,14.179666,4.616350,11.249123,13.721122,16.809595,26.865444,43.3333,15.085086,3484.6548,45.3334,16.124309,3773.0882,2.3300,0.373286,5.2260,231.0,234.0,14.0
10,0.0,10.0,14.053676,3.412040,11.548204,13.950011,16.168706,23.545030,46.0000,17.567756,4005.4483,85.1022,15.902373,3673.4481,0.0000,NaN,0.0000,228.0,231.0,0.0
